In [1]:
import pandas as pd
from neo4j import GraphDatabase
import requests
from pandas.core.internals.construction import arrays_to_mgr
from tqdm import tqdm
import itertools

In [2]:
tqdm.pandas()

In [3]:
driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j', 'neo4j'))

# BISOGNA FARE UNA COSA 4 STAGE:
<ol>
    <li>usare i nomi degli autori presi dall'url dell'immagine e vedere di recuperare quelli non presenti in artgraph;</li>
    <li>ci saranno ancora altri lavori che non vengono presi. Proviamo a prendere il nome dell'autore da <code>artwork_info</code> e di rifare l'operazione
    <li>ci saranno ancora altri lavori che non vengono presi. Se l'autore riscontra delle opere via API, vuol dire che bisogna passare a prendere le info puntuali dell'artwork, quindi prendere il campo <code>url</code> e, infine, provare il matching;
    <li>in ultimo, se l'autore non dà alcun risultato, è molto probabile che ci sia un <b>typo</b> . Va riconosciuto, modificato il tutto e rilanciato
</ol>

Get artworks information

In [4]:
artwork_info = pd.read_csv('WikiArt-Emotions/WikiArt-info.tsv', delimiter='\t')

In [5]:
artwork_info['Artist Info URL'] = artwork_info['Artist Info URL'].progress_apply(
    lambda x: x if 'andrea-del-verrocchio' not in x \
        else '/'.join(x.split('/')[:-1] + ['andrea-del-verrochio']))
artwork_info

100%|██████████| 4119/4119 [00:00<00:00, 228828.70it/s]


,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL
0,58c6237dedc2c9c7dc0de1ae,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,https://uploads3.wikiart.org/00123/images/char...,https://www.wikiart.org/en/charles-courtney-cu...,https://www.wikiart.org/en/charles-courtney-cu...
1,577280dfedc2cb3880f28e76,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,https://uploads1.wikiart.org/images/keith-hari...,https://www.wikiart.org/en/keith-haring/the-ma...,https://www.wikiart.org/en/keith-haring
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie
...,...,...,...,...,...,...,...,...
4114,577287aeedc2cb388007fba9,Color Field Painting,Rupprecht Geiger,OE 260,1957,https://uploads7.wikiart.org/images/rupprecht-...,https://www.wikiart.org/en/rupprecht-geiger/oe...,https://www.wikiart.org/en/rupprecht-geiger
4115,57728001edc2cb3880efddcf,Surrealism,Oscar Dominguez,M&#225;quina de coser electro-sexual,1934,https://uploads4.wikiart.org/images/oscar-domi...,https://www.wikiart.org/en/oscar-dominguez/m-q...,https://www.wikiart.org/en/oscar-dominguez
4116,57728ac7edc2cb3880123cc2,Neo-Expressionism,Georg Baselitz,Female Nude on a Kitchen Chair,1979,https://uploads2.wikiart.org/images/georg-base...,https://www.wikiart.org/en/georg-baselitz/fema...,https://www.wikiart.org/en/georg-baselitz
4117,57728412edc2cb3880fc9ff4,Expressionism,Marie Laurencin,Apollinaire and His Friends,1909,https://uploads0.wikiart.org/images/marie-laur...,https://www.wikiart.org/en/marie-laurencin/apo...,https://www.wikiart.org/en/marie-laurencin


In [6]:
def mod_painting_info(link):
    l = link.split('/')
    l[-2] = 'andrea-del-verrochio'
    return '/'.join(l)

In [7]:
def mod_painting_url(link):
    l = link.split('/')
    l[-2] = 'andrea-del-verrochio'
    return '/'.join(l)

In [8]:
artwork_info['Painting Info URL'] = artwork_info['Painting Info URL'].progress_apply(
    lambda x: x if 'andrea-del-verrocchio' not in x \
        else mod_painting_info(x))
artwork_info['Image URL'] = artwork_info['Image URL'].progress_apply(
    lambda x: x if 'andrea-del-verrocchio' not in x \
        else mod_painting_info(x))
artwork_info

100%|██████████| 4119/4119 [00:00<00:00, 457566.50it/s]


,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL
0,58c6237dedc2c9c7dc0de1ae,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,https://uploads3.wikiart.org/00123/images/char...,https://www.wikiart.org/en/charles-courtney-cu...,https://www.wikiart.org/en/charles-courtney-cu...
1,577280dfedc2cb3880f28e76,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,https://uploads1.wikiart.org/images/keith-hari...,https://www.wikiart.org/en/keith-haring/the-ma...,https://www.wikiart.org/en/keith-haring
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie
...,...,...,...,...,...,...,...,...
4114,577287aeedc2cb388007fba9,Color Field Painting,Rupprecht Geiger,OE 260,1957,https://uploads7.wikiart.org/images/rupprecht-...,https://www.wikiart.org/en/rupprecht-geiger/oe...,https://www.wikiart.org/en/rupprecht-geiger
4115,57728001edc2cb3880efddcf,Surrealism,Oscar Dominguez,M&#225;quina de coser electro-sexual,1934,https://uploads4.wikiart.org/images/oscar-domi...,https://www.wikiart.org/en/oscar-dominguez/m-q...,https://www.wikiart.org/en/oscar-dominguez
4116,57728ac7edc2cb3880123cc2,Neo-Expressionism,Georg Baselitz,Female Nude on a Kitchen Chair,1979,https://uploads2.wikiart.org/images/georg-base...,https://www.wikiart.org/en/georg-baselitz/fema...,https://www.wikiart.org/en/georg-baselitz
4117,57728412edc2cb3880fc9ff4,Expressionism,Marie Laurencin,Apollinaire and His Friends,1909,https://uploads0.wikiart.org/images/marie-laur...,https://www.wikiart.org/en/marie-laurencin/apo...,https://www.wikiart.org/en/marie-laurencin


Get name for trying to do a mapping with ArtGraph

In [9]:
artwork_info['name'] = artwork_info['Image URL'].map(lambda x: '_'.join(x.split('/')[-2:]))

Function that checks if a painting is present in ArtGraph

In [10]:
def is_in_artgraph(param, value, driver):
    base_query = f'match (a:Artwork) where a.{param} = "{value}" return count(distinct a) as num'
    with driver.session(database='neo4j') as session:
        ans = session.run(base_query).data()
    return ans[0]['num']

In [11]:
artwork_info['name_in_artgraph'] = artwork_info.progress_apply(lambda x: is_in_artgraph('name', x['name'], driver), axis=1)

100%|██████████| 4119/4119 [09:28<00:00,  7.24it/s]


In [12]:
artwork_info['artist'] = artwork_info['name'].map(lambda x: x.split('_')[0])
artwork_info['artist_1'] = artwork_info['Artist'].map(lambda x: '-'.join(x.lower().split(' ')))

In [13]:
artwork_info['name'] = artwork_info['name'].map(lambda x: x.split('_')[1])

Taking artworks that are not present in artgraph

In [14]:
unretrieved = artwork_info[artwork_info.name_in_artgraph == 0]

Taking artist name

In [15]:
unretrieved.drop(['Category', 'Artist', 'Year', 'Painting Info URL', 'Artist Info URL', 'name_in_artgraph','Image URL'],
                 axis = 1, inplace=True)

C:\Users\enzos.LAPTOP-RB20LOIL\AppData\Local\Temp\ipykernel_39620\107577158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unretrieved.drop(['Category', 'Artist', 'Year', 'Painting Info URL', 'Artist Info URL', 'name_in_artgraph','Image URL'],


In [16]:
# function that gets all the paintings of a specific artist
def get_paintings_by_artist(artist_name):
    base_query = 'https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl={artist}&json=2'
    return requests.get(base_query.format(artist=artist_name)).json()

# function that gets all painting infos for a specific artwork
def get_painting_info(painting_id: int):
    base_query = 'https://www.wikiart.org/en/App/Painting/ImageJson/{painting}'
    return requests.get(base_query.format(painting=painting_id)).json()

# function that gets all url unmatched artworks with respect to wikiart artwork info
def retrieve_artworks_by_url(artist_name, metadata):
    paintings = get_paintings_by_artist(artist_name)
    paintings = map(lambda x: x['contentId'], paintings)
    paintings = list(map(lambda x: f"{get_painting_info(x)['url']}.jpg", paintings))
    return metadata[metadata['name'].isin(paintings)].name.tolist()

# function that gets all unmatched artworks with respect to basic info got by searching just paintings of spceific artwork
def retrieve_artworks(artist_name, metadata):
    paintings = get_paintings_by_artist(artist_name)
    titles = list(map(lambda x: x['title'], paintings))
    names = list(map(lambda x: x['image'].split('/')[-1][:-10], paintings))
    return metadata[(metadata['Title'].isin(titles)) | (metadata['name'].isin(names))].name.tolist()

Group paintings by artist

In [17]:
grouped = unretrieved.groupby(['artist', 'artist_1'])[['Title', 'name']].apply(lambda x: x.values.tolist()).reset_index(name='metadata')

Retrieve artworks following **1**.

In [18]:
retrieved = grouped.progress_apply(lambda x: retrieve_artworks(x.artist, pd.DataFrame(x.metadata, columns=['Title', 'name'])), axis=1)

100%|██████████| 501/501 [02:08<00:00,  3.90it/s]


In [19]:
total = unretrieved.groupby(['artist', 'artist_1'])['name'].apply(list).reset_index(name='names')

placing the first retrieved artwroks

In [20]:
total['ret'] = retrieved

Making comparison and underline unretrieved artworks

In [21]:
total['difference'] = total.apply(lambda x: list(set(x.names) - set(x.ret)), axis=1)
total

,artist,artist_1,names,ret,difference
0,aaron-siskind,aaron-siskind,"[feet-102-1957.jpg, jerome-arizona-1949.jpg]","[feet-102-1957.jpg, jerome-arizona-1949.jpg]",[]
1,abraham-manievich,abraham-manievich,[destruction-of-the-ghetto-kiev-1919.jpg],[destruction-of-the-ghetto-kiev-1919.jpg],[]
2,ad-reinhardt,ad-reinhardt,[study-for-a-painting-1938-1.jpg],[study-for-a-painting-1938-1.jpg],[]
3,adolph-menzel,adolph-menzel,[afternoon-in-the-tuileries-gardens-1867(1).jp...,[afternoon-in-the-tuileries-gardens-1867(1).jp...,[]
4,adriaen-brouwer,adriaen-brouwer,"[youth-making-a-face.jpg, fumatore.jpg, the-bi...","[youth-making-a-face.jpg, fumatore.jpg, the-bi...",[]
...,...,...,...,...,...
496,wolfgang-paalen,wolfgang-paalen,[genius-of-the-species-1938.jpg],[genius-of-the-species-1938.jpg],[]
497,wyndham-lewis,wyndham-lewis,[the-crowd-1915.jpg],[the-crowd-1915.jpg],[]
498,xul-solar,xul-solar,"[reptil-que-sube-1920.jpg, pegaso-de-sol-1922....","[reptil-que-sube-1920.jpg, pegaso-de-sol-1922....",[]
499,yannoulis-chalepas,yannoulis-chalepas,"[mercury-venus-and-pegasus-1933.jpg, satyr-pla...","[mercury-venus-and-pegasus-1933.jpg, satyr-pla...",[]


In [22]:
total.difference.map(len).sum()

51

In [23]:
paintings_still_unretrieved = list(itertools.chain(*total.difference.values.tolist()))
artwork_info['api_v1_artist'] = artwork_info.progress_apply(lambda x: 1 if x['name'] not in paintings_still_unretrieved and not x.name_in_artgraph else 0, axis=1)

100%|██████████| 4119/4119 [00:00<00:00, 65882.89it/s]


Attualmente ce ne mancano 51

# STAGE 2: USARE L'ALTRO NOME

In [24]:
unretrieved = unretrieved[unretrieved.name.isin(paintings_still_unretrieved)]

In [25]:
grouped = unretrieved.groupby(['artist', 'artist_1'])[['Title', 'name']].apply(lambda x: x.values.tolist()).reset_index(name='metadata')
retrieved = grouped.progress_apply(lambda x: retrieve_artworks(x.artist_1, pd.DataFrame(x.metadata, columns=['Title', 'name'])), axis=1)

100%|██████████| 28/28 [00:07<00:00,  3.71it/s]


In [26]:
total = unretrieved.groupby(['artist', 'artist_1'])['name'].apply(list).reset_index(name='names')
total['ret'] = retrieved
total['difference'] = total.apply(lambda x: list(set(x.names) - set(x.ret)), axis=1)

In [27]:
total

,artist,artist_1,names,ret,difference
0,andrea-del-verrochio,andrea-del-verrocchio,"[lorenzo-de-medici-1480.jpg, david-1475.jpg, p...",[],"[lorenzo-de-medici-1480.jpg, the-battle-of-pyd..."
1,anton-raphael-mengs,anton-raphael-mengs,[carlos-iii-1761.jpg],[],[carlos-iii-1761.jpg]
2,benjamin-west,benjamin-west,[the-treaty-of-penn-with-the-indians-1772.jpg],[],[the-treaty-of-penn-with-the-indians-1772.jpg]
3,boris-gregoriev,boris-grigoriev,[portrait-of-vsevolod-meyerhold-1916.jpg],[portrait-of-vsevolod-meyerhold-1916.jpg],[]
4,caspar-david-friedrich,caspar-david-friedrich,[chalk-cliffs-on-r-gen-1818.jpg],[],[chalk-cliffs-on-r-gen-1818.jpg]
5,francisco-goya,francisco-goya,[saturn-devouring-one-of-his-children-1823.jpg],[],[saturn-devouring-one-of-his-children-1823.jpg]
6,george-bousianis,george-bouzianis,[kefali-andra-1915.jpg],[kefali-andra-1915.jpg],[]
7,j-zsef-rippl-r-nai,jozsef-rippl-ronai,[uncle-piacsek-in-front-of-the-black-sideboard...,[uncle-piacsek-in-front-of-the-black-sideboard...,[]
8,jean-auguste-dominique-ingres,jean-auguste-dominique-ingres,[portrait-of-napoléon-on-the-imperial-throne-1...,[],[portrait-of-napoléon-bonaparte-the-first-coun...
9,john-henry-twachtman,john-henry-twachtman,[landscape.jpg],[landscape.jpg],[]


In [28]:
total[total.difference.map(len) != 0]

,artist,artist_1,names,ret,difference
0,andrea-del-verrochio,andrea-del-verrocchio,"[lorenzo-de-medici-1480.jpg, david-1475.jpg, p...",[],"[lorenzo-de-medici-1480.jpg, the-battle-of-pyd..."
1,anton-raphael-mengs,anton-raphael-mengs,[carlos-iii-1761.jpg],[],[carlos-iii-1761.jpg]
2,benjamin-west,benjamin-west,[the-treaty-of-penn-with-the-indians-1772.jpg],[],[the-treaty-of-penn-with-the-indians-1772.jpg]
4,caspar-david-friedrich,caspar-david-friedrich,[chalk-cliffs-on-r-gen-1818.jpg],[],[chalk-cliffs-on-r-gen-1818.jpg]
5,francisco-goya,francisco-goya,[saturn-devouring-one-of-his-children-1823.jpg],[],[saturn-devouring-one-of-his-children-1823.jpg]
8,jean-auguste-dominique-ingres,jean-auguste-dominique-ingres,[portrait-of-napoléon-on-the-imperial-throne-1...,[],[portrait-of-napoléon-bonaparte-the-first-coun...
10,john-singleton-copley,john-singleton-copley,[the-major-peirson-s-death-1784.jpg],[],[the-major-peirson-s-death-1784.jpg]
13,li-chevalier,li-chevalier,"[arrachement.jpg, 211.jpg]",[],"[arrachement.jpg, 211.jpg]"
15,max-herman-maxy,m.-h.-maxy,"[nude-with-veil-1922.jpg, nude-1924.jpg, human...",[],"[nude-with-veil-1922.jpg, human-construction-1..."
16,otto-mueller,otto-mueller,[otto-mueller-t-nzerin-mit-schleier-von-einem-...,[],[otto-mueller-bauerngeh-ft-in-frankreich-ca191...


# STAGE **3**
## RECURSIVE QUERY BASED TO URL

In [29]:
paintings_still_unretrieved = list(itertools.chain(*total.difference.values.tolist()))


In [30]:
artwork_info['api_v1_artist_1'] = artwork_info.progress_apply(lambda x: 1 if x['name'] not in paintings_still_unretrieved and not x.api_v1_artist else 0, axis=1)

100%|██████████| 4119/4119 [00:00<00:00, 33872.38it/s]


In [31]:
unretrieved = unretrieved[unretrieved.name.isin(paintings_still_unretrieved)]

In [33]:
grouped = unretrieved.groupby(['artist', 'artist_1'])[['Title', 'name']].apply(lambda x: x.values.tolist()).reset_index(name='metadata')
retrieved = grouped.progress_apply(lambda x: retrieve_artworks_by_url(x.artist, pd.DataFrame(x.metadata, columns=['Title', 'name'])), axis=1)

100%|██████████| 17/17 [10:15<00:00, 36.22s/it]


In [34]:
total = unretrieved.groupby(['artist', 'artist_1'])['name'].apply(list).reset_index(name='names')
total['ret'] = retrieved
total['difference'] = total.apply(lambda x: list(set(x.names) - set(x.ret)), axis=1)

In [35]:
total[total.difference.map(len) != 0]

,artist,artist_1,names,ret,difference
0,andrea-del-verrochio,andrea-del-verrocchio,"[lorenzo-de-medici-1480.jpg, david-1475.jpg, p...",[],"[lorenzo-de-medici-1480.jpg, the-battle-of-pyd..."
3,caspar-david-friedrich,caspar-david-friedrich,[chalk-cliffs-on-r-gen-1818.jpg],[],[chalk-cliffs-on-r-gen-1818.jpg]
4,francisco-goya,francisco-goya,[saturn-devouring-one-of-his-children-1823.jpg],[],[saturn-devouring-one-of-his-children-1823.jpg]
5,jean-auguste-dominique-ingres,jean-auguste-dominique-ingres,[portrait-of-napoléon-on-the-imperial-throne-1...,[portrait-of-napoléon-bonaparte-the-first-coun...,[portrait-of-napoléon-on-the-imperial-throne-1...
8,li-chevalier,li-chevalier,"[arrachement.jpg, 211.jpg]",[],"[arrachement.jpg, 211.jpg]"
9,max-herman-maxy,m.-h.-maxy,"[nude-with-veil-1922.jpg, nude-1924.jpg, human...",[],"[nude-with-veil-1922.jpg, human-construction-1..."
10,otto-mueller,otto-mueller,[otto-mueller-t-nzerin-mit-schleier-von-einem-...,[],[otto-mueller-bauerngeh-ft-in-frankreich-ca191...
11,p-ricl-s-pantazis,pericles-pantazis-,"[mauvaise-recette.jpg, supreme-court-1880.jpg,...",[],"[landscape.jpg, supreme-court-1880.jpg, mauvai..."
12,peter-paul-rubens,peter-paul-rubens,[rape-of-the-daughters-of-leucippus.jpg],[],[rape-of-the-daughters-of-leucippus.jpg]
13,pierre-narcisse-guerin,jean-auguste-dominique-ingres,[the-portrait-of-the-chouan-rochejaquelin.jpg],[],[the-portrait-of-the-chouan-rochejaquelin.jpg]


# STAGE 4

In [36]:
paintings_still_unretrieved = list(itertools.chain(*total.difference.values.tolist()))
artwork_info['api_v1_url'] = artwork_info.progress_apply(lambda x: 1 if x['name'] not in paintings_still_unretrieved and not x.api_v1_artist_1 else 0, axis=1)

100%|██████████| 4119/4119 [00:00<00:00, 11072.53it/s]


In [37]:
unretrieved = unretrieved[unretrieved.name.isin(paintings_still_unretrieved)]

In [38]:
def get_painting(id: str):
    base = 'https://www.wikiart.org/en/api/2/Painting?id={id}'
    return requests.get(base.format(id=id)).json()

In [39]:
unretrieved['api_v2'] = unretrieved.ID.progress_apply(lambda x: 'title' in get_painting(x))
unretrieved

100%|██████████| 25/25 [00:06<00:00,  3.89it/s]


,ID,Title,name,artist,artist_1,api_v2
41,57728204edc2cb3880f63d8a,Clocher du Village,clocher-du-village-1958.jpg,syed-haider-raza,s.-h.-raza,True
63,59839e9cedc2c9658c2f54cd,Arrachement,arrachement.jpg,li-chevalier,li-chevalier,False
116,57727ce5edc2cb3880e5fb4f,Mauvaise Recette,mauvaise-recette.jpg,p-ricl-s-pantazis,pericles-pantazis-,True
218,57727882edc2cb3880d81610,Landscape,landscape.jpg,john-henry-twachtman,john-henry-twachtman,True
703,5772779bedc2cb3880d4fdcf,Lorenzo de Medici,lorenzo-de-medici-1480.jpg,andrea-del-verrochio,andrea-del-verrocchio,False
790,59839e9cedc2c9658c2f54cb,211,211.jpg,li-chevalier,li-chevalier,False
838,577280c3edc2cb3880f26379,Nude with Veil,nude-with-veil-1922.jpg,max-herman-maxy,m.-h.-maxy,True
858,57726e7aedc2cb3880b744fa,Primavera,primavera-1478(1).jpg,sandro-botticelli,sandro-botticelli,True
953,577280c3edc2cb3880f26359,Nude,nude-1924.jpg,max-herman-maxy,m.-h.-maxy,True
1283,57cd005cedc2c99704d737cb,Chalk Cliffs on R&#252;gen,chalk-cliffs-on-r-gen-1818.jpg,caspar-david-friedrich,caspar-david-friedrich,False


In [40]:
artwork_info['api_v2'] = artwork_info.progress_apply(lambda x: 1 if x['name'] not in unretrieved[unretrieved.api_v2==False]['name'].tolist() and not x.api_v1_url else 0, axis=1)

100%|██████████| 4119/4119 [00:03<00:00, 1074.26it/s]


In [41]:
unretrieved[unretrieved.api_v2==False]

,ID,Title,name,artist,artist_1,api_v2
63,59839e9cedc2c9658c2f54cd,Arrachement,arrachement.jpg,li-chevalier,li-chevalier,False
703,5772779bedc2cb3880d4fdcf,Lorenzo de Medici,lorenzo-de-medici-1480.jpg,andrea-del-verrochio,andrea-del-verrocchio,False
790,59839e9cedc2c9658c2f54cb,211,211.jpg,li-chevalier,li-chevalier,False
1283,57cd005cedc2c99704d737cb,Chalk Cliffs on R&#252;gen,chalk-cliffs-on-r-gen-1818.jpg,caspar-david-friedrich,caspar-david-friedrich,False
1897,5772779bedc2cb3880d4fd9d,David,david-1475.jpg,andrea-del-verrochio,andrea-del-verrocchio,False
2095,5772779bedc2cb3880d4fe0f,Portrait of a Woman,portrait-of-a-woman.jpg,andrea-del-verrochio,andrea-del-verrocchio,False
2836,5772779bedc2cb3880d4fe5f,The Battle of Pydna,the-battle-of-pydna.jpg,andrea-del-verrochio,andrea-del-verrocchio,False


In [42]:
unretrieved[unretrieved.api_v2==False].to_csv('still_unretrieved.csv')

In [43]:
artwork_info

,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL,name,name_in_artgraph,artist,artist_1,api_v1_artist,api_v1_artist_1,api_v1_url,api_v2
0,58c6237dedc2c9c7dc0de1ae,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,https://uploads3.wikiart.org/00123/images/char...,https://www.wikiart.org/en/charles-courtney-cu...,https://www.wikiart.org/en/charles-courtney-cu...,in-the-luxembourg-garden-1889.jpg,1,charles-courtney-curran,charles-courtney-curran,1,1,1,1
1,577280dfedc2cb3880f28e76,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,https://uploads1.wikiart.org/images/keith-hari...,https://www.wikiart.org/en/keith-haring/the-ma...,https://www.wikiart.org/en/keith-haring,the-marriage-of-heaven-and-hell-1984.jpg,1,keith-haring,keith-haring,1,1,1,1
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai,uncle-piacsek-in-front-of-the-black-sideboard-...,0,j-zsef-rippl-r-nai,jozsef-rippl-ronai,0,1,1,1
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller,monk-for-the-play-mazeppa-1920.jpg,0,vadym-meller,vadym-meller,1,1,1,1
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie,the-defence-of-sarago-a.jpg,0,david-wilkie,david-wilkie,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,577287aeedc2cb388007fba9,Color Field Painting,Rupprecht Geiger,OE 260,1957,https://uploads7.wikiart.org/images/rupprecht-...,https://www.wikiart.org/en/rupprecht-geiger/oe...,https://www.wikiart.org/en/rupprecht-geiger,oe-260-1957.jpg,1,rupprecht-geiger,rupprecht-geiger,1,1,1,1
4115,57728001edc2cb3880efddcf,Surrealism,Oscar Dominguez,M&#225;quina de coser electro-sexual,1934,https://uploads4.wikiart.org/images/oscar-domi...,https://www.wikiart.org/en/oscar-dominguez/m-q...,https://www.wikiart.org/en/oscar-dominguez,m-quina-de-coser-electro-sexual-1934.jpg,1,oscar-dominguez,oscar-dominguez,1,1,1,1
4116,57728ac7edc2cb3880123cc2,Neo-Expressionism,Georg Baselitz,Female Nude on a Kitchen Chair,1979,https://uploads2.wikiart.org/images/georg-base...,https://www.wikiart.org/en/georg-baselitz/fema...,https://www.wikiart.org/en/georg-baselitz,female-nude-on-a-kitchen-chair-1979.jpg,1,georg-baselitz,georg-baselitz,1,1,1,1
4117,57728412edc2cb3880fc9ff4,Expressionism,Marie Laurencin,Apollinaire and His Friends,1909,https://uploads0.wikiart.org/images/marie-laur...,https://www.wikiart.org/en/marie-laurencin/apo...,https://www.wikiart.org/en/marie-laurencin,apollinaire-and-his-friends-1909.jpg,1,marie-laurencin,marie-laurencin,1,1,1,1


In [ ]:
artwork_info.to_csv('artwork_info_sources.csv')